In [1]:
def calculate_cvss_base_score(av, ac, pr, ui, s, c, i, a):
    # Metric values
    av_values = {'N': 0.85, 'A': 0.62, 'L': 0.55, 'P': 0.2}
    ac_values = {'L': 0.77, 'H': 0.44}
    pr_values_unchanged = {'N': 0.85, 'L': 0.62, 'H': 0.27}
    pr_values_changed = {'N': 0.85, 'L': 0.68, 'H': 0.5}
    ui_values = {'N': 0.85, 'R': 0.62}
    s_values = {'U': 1.0, 'C': 1.0}
    impact_values = {'N': 0.0, 'L': 0.22, 'H': 0.56}

    # Extract values
    av_score = av_values[av]
    ac_score = ac_values[ac]
    pr_score = pr_values_unchanged[pr] if s == 'U' else pr_values_changed[pr]
    ui_score = ui_values[ui]
    c_score = impact_values[c]
    i_score = impact_values[i]
    a_score = impact_values[a]

    # Calculate Exploitability
    exploitability = 8.22 * av_score * ac_score * pr_score * ui_score

    # Calculate Impact
    impact = 1 - ((1 - c_score) * (1 - i_score) * (1 - a_score))
    if s == 'U':
        impact *= 6.42
    else:  # Scope is 'C'
        impact = 7.52 * (impact - 0.029) - 3.25 * (impact - 0.02) ** 15

    # Calculate Base Score
    if impact <= 0:
        base_score = 0
    else:
        if s == 'U':
            base_score = min(exploitability + impact, 10)
        else:
            base_score = min(1.08 * (exploitability + impact), 10)

    # Round to 1 decimal place
    base_score = round(base_score, 1)

    return base_score

def get_severity_level(score):
    if score == 0.0:
        return "None"
    elif 0.1 <= score <= 3.9:
        return "Low"
    elif 4.0 <= score <= 6.9:
        return "Medium"
    elif 7.0 <= score <= 8.9:
        return "High"
    elif 9.0 <= score <= 10.0:
        return "Critical"
    else:
        return "Unknown"

def main():
    while True:
        # Prompt user for inputs
        av = input("Enter Attack Vector (N/A/L/P): ").strip().upper()
        ac = input("Enter Attack Complexity (L/H): ").strip().upper()
        pr = input("Enter Privileges Required (N/L/H): ").strip().upper()
        ui = input("Enter User Interaction (N/R): ").strip().upper()
        s = input("Enter Scope (U/C): ").strip().upper()
        c = input("Enter Confidentiality Impact (N/L/H): ").strip().upper()
        i = input("Enter Integrity Impact (N/L/H): ").strip().upper()
        a = input("Enter Availability Impact (N/L/H): ").strip().upper()

        # Validate inputs
        valid_values = {
            'av': {'N', 'A', 'L', 'P'},
            'ac': {'L', 'H'},
            'pr': {'N', 'L', 'H'},
            'ui': {'N', 'R'},
            's': {'U', 'C'},
            'c': {'N', 'L', 'H'},
            'i': {'N', 'L', 'H'},
            'a': {'N', 'L', 'H'}
        }

        for key, valid_set in valid_values.items():
            value = locals()[key]
            if value not in valid_set:
                print(f"Invalid value for {key}. Please enter a valid value.")
                return

        # Calculate and display CVSS Base Score
        base_score = calculate_cvss_base_score(av, ac, pr, ui, s, c, i, a)
        severity_level = get_severity_level(base_score)
        print(f'The CVSS Base Score is: {base_score}')
        print(f'Severity Level: {severity_level}')

        # Ask if the user wants to calculate another score
        another = input("Calculate another Score? (yes/no): ").strip().lower()
        if another != 'yes':
            print("Exit the program.")
            break

if __name__ == "__main__":
    main()


Enter Attack Vector (N/A/L/P): P
Enter Attack Complexity (L/H): H
Enter Privileges Required (N/L/H): H
Enter User Interaction (N/R): N
Enter Scope (U/C): U
Enter Confidentiality Impact (N/L/H): H
Enter Integrity Impact (N/L/H): H
Enter Availability Impact (N/L/H): H
The CVSS Base Score is: 6.0
Severity Level: Medium
Calculate another CVSS Base Score? (yes/no): no
Exit the program.
